## libs

In [251]:
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as m
#from scipy.interpolate import interp1d
#import scipy.optimize
import scipy.integrate as integrate

## constants

In [335]:
freq_ion = 3.29 * 10**15 #Herz
freq_max = 5 * 10**16 #Herz
alpha = 2.59E-13 #cm^3 per sec
temperature = 40000 #Kelvins
radius_star = 15 * 6.957E2 / (206264.8 * 149.6* 10**6)#pc
r_inner = 0.05 #pc
step = 0.0001 #pc

c = 299792458.0E2 #cm per sec , light speed
h = 6.626070040812E-27 #erg*sec, Planck constant
k = 1.380648528E-16 #erg/K, Boltzmann constant
nH_total = 100 #cm^(-3)

In [337]:
freq = np.linspace (freq_ion, freq_max, 300)
freq_tau = pd.DataFrame(data = {"freq" : freq, "tau" : np.full(300, 0)})
result = pd.DataFrame(data = {"r" : [r_inner], "n_e" : [100]})
r_previous = r_inner
r_current = r_inner + step 
check = True

## functions

In [338]:
def brightness(freq):
    """mean brightness"""
    return 2 * h * freq**3 / c**2 / (np.exp(h * freq / (k * temperature)) - 1) 
    
def sigma(freq):
    """return photoionization cross section"""
    return 6.3E-12 * np.power (freq_ion / freq, 3)

def tau_plus(freq, nH_neutral):
    """opacity on distance r"""
    return nH_neutral * sigma(freq) * step * (206264.8 * 149.6 * 10**12)

def intensity(freq,r):
    """mean intensity Iν as a function of frequency"""
    return 0.25 * brightness(freq) * (radius_star / r)**2 * np.exp(-tau(freq, r))

In [ ]:
n_e = nH_total
while np.abs(n_e)>0.001:
    check = True
    iteration = 0
    while check:
        PHI = integrate.trapz(m.pi * brightness(freq)  * (radius_star / r_current)**2 * np.exp(-freq_tau["tau"]) * sigma(freq) / (h * freq), freq)
        ksi = PHI / (n_e * alpha)
        nH_neutral = nH_total / (1 + ksi)
        nH_ionized = nH_total * ksi / (1 + ksi)
        n_e_new = np.sqrt(n_e * nH_ionized)
        nH_neutral = nH_total - n_e_new
        freq_tau["tau"] = tau_plus(freq, nH_neutral)
        if np.abs(n_e_new/n_e - 1) < 0.1E-5:
            n_e = n_e_new
            result = result.append(pd.DataFrame(data = {"r" : [r_current], "n_e" : [n_e]}), ignore_index = True)
            check = False
        else:
            n_e = n_e_new
            nH_neutral = nH_total - n_e
            iteration +=1
    print(iteration)        
    r_previous = r_current
    r_current += step    

In [340]:
result

,r,n_e
0,0.0500,1.000000e+02
1,0.0501,1.000000e+02
2,0.0502,9.999999e+01
3,0.0503,9.999999e+01
4,0.0504,9.999999e+01
5,0.0505,9.999999e+01
6,0.0506,9.999999e+01
7,0.0507,9.999999e+01
8,0.0508,9.999999e+01
9,0.0509,9.999999e+01


In [297]:
check = True
while check:
    PHI = integrate.trapz(m.pi * brightness(freq)  * (radius_star / r_current)**2 * np.exp(-freq_tau["tau"]) * sigma(freq) / (h * freq), freq)
    ksi = PHI / (n_e * alpha)
    nH_neutral = nH_total / (1 + ksi)
    nH_ionized = nH_total * ksi / (1 + ksi)
    n_e_new = np.sqrt(n_e * nH_ionized)
    nH_neutral = nH_total - n_e_new
    freq_tau["tau"] = tau_plus(freq, nH_neutral)
    if np.abs(n_e_new/n_e - 1) < 0.1E-3:
        n_e = n_e_new
        result = result.append(pd.DataFrame(data = {"r" : [r_current], "n_e" : [n_e]}), ignore_index = True)
        check = False
    else:
        n_e = n_e_new
        nH_neutral = nH_total - n_e

In [298]:
result

,r,n_e
0,0.0500,100.000000
1,0.0501,99.999996
2,0.0502,99.999993
